In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
import re

#filter(None, re.split('(-|:|am|pm)', '30'))


#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [3]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [5]:
question_class = 'q2'
fname = '..//dataset//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of comments', len(content)
f.close()

length of comments 478


In [6]:
def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def num_there(s):
    return any(i.isdigit() for i in s)

def process_corpus(x1, day_senswords, time_senswords, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_days = []
    doc_time = []
    doc_other = []
    doc_multi = []
    doc_nn = []
    for x in x1:
        if 'no improvement' in x or 'respondent' in x:
            doc_noimprove.append(x)
        else:
            sents = sent_tokenize(x)
            if len(sents) > 1:
                doc_multi.append(x)
            else:
                nn_list = []
                sen = sents[0]
                word_list = nltk.word_tokenize(sen)
                if not set(word_list).isdisjoint(day_senswords):
                    doc_days.append(sen)
                else:
                    clean_word_list = []
                    for word in word_list:
                        clean_word_list = clean_word_list + filter(None, re.split('(-|:|am|pm)', word)) 
                    if 'am' in clean_word_list or 'pm' in clean_word_list or num_there(clean_word_list):
                        doc_time.append(sen)
                        doc_nn.append(clean_word_list)
                    else:
                        doc_other.append(sen)

    return doc_days, [doc_time,doc_nn], doc_other
    #return doc_noimprove, [doc_nn, nn_extracted], doc_other, doc_multi
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]

for punctuation in punctuation_list:
    stop_words.append(punctuation)

day_sense = ['sunday', 'sundays', 'weekend', 'weekends', 'holidays', 'holiday']
time_sense = ['am', 'pm',]
t1, t2, t3 = process_corpus(content, day_sense, time_sense, stop_words)




In [26]:
def extract_am(word_list):
    if 'am' in word_list:
        amidxs = [i for i,val in enumerate(word_list) if val=='am']
        time_nn = []
        #print len(amidxs)
        for amidx in amidxs:
            time_slot = []
            idx_start = max(amidx-2, 0)
            for ix in range(idx_start, amidx):
                if word_list[ix].isdigit():
                    time_slot.append(word_list[ix])
            if len(time_slot) == 2:
                time_value = int(time_slot[0])+int(time_slot[1])/60.0
                time_nn.append(time_value)
            if len(time_slot) == 1:
                time_nn.append(int(time_slot[0]))
        if time_nn:
            return min(time_nn)
        else:
            return None

def extract_pm(word_list):
    if 'pm' in word_list:
        amidxs = [i for i,val in enumerate(word_list) if val=='pm']
        time_nn = []
        #print len(amidxs)
        for amidx in amidxs:
            time_slot = []
            idx_start = max(amidx-2, 0)
            for ix in range(idx_start, amidx):
                if word_list[ix].isdigit():
                    time_slot.append(word_list[ix])
            if len(time_slot) == 2:
                time_value = int(time_slot[0])+int(time_slot[1])/60.0
                time_nn.append(time_value)
            if len(time_slot) == 1:
                time_nn.append(int(time_slot[0]))
        if time_nn:
            print word_list
            return max(time_nn)
        else:
            return None
for sen in t2[1]:
    print extract_pm(sen)

None
['service', 'center', 'should', 'be', 'open', 'till', '7', 'pm', '.']
7
['if', 'a', 'customer', 'gives', 'car', 'for', 'servicing', 'then', 'they', 'should', 'do', 'service', 'properly', 'and', 'extend', 'the', 'dealership', 'evening', 'closing', 'time', 'after', '6', 'pm', '.']
6
None
['they', 'open', 'by', '10', 'am', 'in', 'the', 'morning', 'instead', 'they', 'should', 'open', 'by', '9', 'am', 'and', 'close', 'by', '7', 'pm', '.']
7
None
None
['the', 'closing', 'time', 'of', 'dealership', 'should', 'increase', 'after', '7', '00', 'pm']
7.0
None
None
None
['in', 'the', 'morning', 'dealership', 'should', 'open', 'by', '8', 'am', 'and', 'by', '5', 'pm', 'it', 'should', 'be', 'closed', '.']
5
None
['center', 'should', 'be', 'open', 'at', '9', 'am', 'and', 'close', 'by', '7', 'pm', '.']
7
None
['dealership', 'should', 'keep', 'open', 'till', '10', 'pm', '.']
10
None
None
['dealership', 'opening', 'closing', 'time', 'should', 'keep', 'the', 'opening', 'till', '8', 'pm']
8
None
['they

# Split Large Cluster